# AB Testi ile BiddingYöntemlerinin Dönüşümünün Karşılaştırılması

#####################################################
### İş Problemi


Facebook kısa süre önce mevcut "maximumbidding" adı verilen teklif verme türüne alternatif olarak yeni bir teklif türü olan "average bidding"’i tanıttı. 

Müşterilerimizden biri olan xyz.com, bu yeni özelliği test etmeye karar verdi ve averagebidding'in maximumbidding'den daha fazla dönüşüm getirip getirmediğini anlamak için bir A/B testi yapmak istiyor. 

A/B testi 1 aydır devam ediyor ve xyz.com şimdi sizden bu A/B testinin sonuçlarını analiz etmenizi bekliyor. xyz.com için nihai başarı ölçütü Purchase'dır. Bu nedenle, istatistiksel testler için Purchase metriğine odaklanılmalıdır.

#####################################################

In [11]:
#####################################################
# Veri Seti Hikayesi

# Bir firmanın web site bilgilerini içeren bu veri setinde 
# kullanıcıların gördükleri ve tıkladıkları reklam sayıları gibi bilgilerin yanı sıra 
# buradan gelen kazanç bilgileri yer almaktadır.

# Kontrol ve Test grubu olmak üzere iki ayrı veri seti vardır. 
# Bu veri setleriab_testing.xlsx excel’inin ayrı sayfalarında yer almaktadır. 
# Kontrol grubuna Maximum Bidding, test grubuna AverageBidding uygulanmıştır.


# Değişkenler;

# impression: Reklam görüntüleme sayısı
# Click: Görüntülenen reklama tıklama sayısı
# Purchase: Tıklanan reklamlar sonrası satın alınan ürün sayısı
# Earning: Satın alınan ürünler sonrası elde edilen kazanç

In [12]:
# To Do List;

# Görev 1:  Veriyi Hazırlama ve Analiz Etme

# Görev 2:  A/B Testinin Hipotezinin Tanımlanması

# GÖREV 3: Hipotez Testinin Gerçekleştirilmesi

In [13]:
# Kütüphanelerin import edilmesi;

import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import shapiro, levene, ttest_ind
import et_analysis as et

# Pandas ayarlamalarının yapılması;

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### Görev 1:

In [14]:
# Test ve kontrol verilerimizi okutalım;

df_control = pd.read_excel("datasets/ab_testing.xlsx" , sheet_name="Control Group")
df_test = pd.read_excel("datasets/ab_testing.xlsx" , sheet_name="Test Group")

In [15]:
# Kontrol verilerine  hızlı bir bakış atalım;

et.check_df(df_control, display_=True)

##################### Shape #####################
(40, 4)
##################### Types #####################


,DataType
Impression,float64
Click,float64
Purchase,float64
Earning,float64


##################### Head #####################


,Impression,Click,Purchase,Earning
0,82529.46,6090.08,665.21,2311.28
1,98050.45,3382.86,315.08,1742.81
2,82696.02,4167.97,458.08,1797.83
3,109914.40,4910.88,487.09,1696.23
4,108457.76,5987.66,441.03,1543.72


##################### Tail #####################


,Impression,Click,Purchase,Earning
35,132064.22,3747.16,551.07,2256.98
36,86409.94,4608.26,345.05,1781.36
37,123678.93,3649.07,476.17,2187.72
38,101997.49,4736.35,474.61,2254.56
39,121085.88,4285.18,590.41,1289.31


##################### NA #####################


,# of N/A value
Impression,0
Click,0
Purchase,0
Earning,0


##################### Quantiles #####################


,0.00,0.01,0.05,0.25,0.50,0.75,0.95,0.99,1.00
Impression,45475.94,58072.04,79412.02,85726.69,99790.70,115212.82,132950.53,143105.79,147539.34
Click,2189.75,2535.12,3367.48,4124.30,5001.22,5923.80,7374.36,7761.80,7959.13
Purchase,267.03,285.77,328.66,470.10,531.21,637.96,748.27,790.19,801.80
Earning,1253.99,1267.76,1329.58,1685.85,1975.16,2119.80,2318.53,2481.31,2497.30


In [16]:
# Test verilerine  hızlı bir bakış atalım;

et.check_df(df_test, display_=True)

##################### Shape #####################
(40, 4)
##################### Types #####################


,DataType
Impression,float64
Click,float64
Purchase,float64
Earning,float64


##################### Head #####################


,Impression,Click,Purchase,Earning
0,120103.50,3216.55,702.16,1939.61
1,134775.94,3635.08,834.05,2929.41
2,107806.62,3057.14,422.93,2526.24
3,116445.28,4650.47,429.03,2281.43
4,145082.52,5201.39,749.86,2781.70


##################### Tail #####################


,Impression,Click,Purchase,Earning
35,79234.91,6002.21,382.05,2277.86
36,130702.24,3626.32,449.82,2530.84
37,116481.87,4702.78,472.45,2597.92
38,79033.83,4495.43,425.36,2595.86
39,102257.45,4800.07,521.31,2967.52


##################### NA #####################


,# of N/A value
Impression,0
Click,0
Purchase,0
Earning,0


##################### Quantiles #####################


,0.00,0.01,0.05,0.25,0.50,0.75,0.95,0.99,1.00
Impression,79033.83,79112.25,83150.50,112691.97,119291.30,132050.58,153178.69,158245.26,158605.92
Click,1836.63,2005.23,2600.36,3376.82,3931.36,4660.50,5271.19,6012.88,6019.70
Purchase,311.63,325.21,356.70,444.63,551.36,699.86,854.21,876.58,889.91
Earning,1939.61,1942.78,2080.98,2280.54,2544.67,2761.55,2931.31,3091.94,3171.49


In [20]:
# concat metodunu kullanarak kontrol ve test grubu verilerini birleştiriniz.

df_control["group"] = "control"
df_test["group"] = "test"

df = pd.concat([df_control,df_test], axis=0,ignore_index=False)
display(df.head())
print(end="\n\n\n")
display(df.tail())

,Impression,Click,Purchase,Earning,group
0,82529.46,6090.08,665.21,2311.28,control
1,98050.45,3382.86,315.08,1742.81,control
2,82696.02,4167.97,458.08,1797.83,control
3,109914.40,4910.88,487.09,1696.23,control
4,108457.76,5987.66,441.03,1543.72,control


,Impression,Click,Purchase,Earning,group
35,79234.91,6002.21,382.05,2277.86,test
36,130702.24,3626.32,449.82,2530.84,test
37,116481.87,4702.78,472.45,2597.92,test
38,79033.83,4495.43,425.36,2595.86,test
39,102257.45,4800.07,521.31,2967.52,test


### Görev 2:

In [21]:
# Hipotezi tanımlayalım;

# H0 : M1 = M2 (Kontrol grubu ve test grubu satın alma ortalamaları arasında fark yoktur.)
# H1 : M1!= M2 (Kontrol grubu ve test grubu satın alma ortalamaları arasında fark vardır.)

In [22]:
# Kontrol ve test grubu için purchase(kazanç) ortalamalarını analiz ediniz

df.groupby("group").agg({"Purchase": "mean"})

,Purchase
group,
control,550.89
test,582.11


### GÖREV 3:

In [ ]:
# Hipotez testi yapılmadan önce varsayım kontrollerini yapalım.
# Bunlar Normallik Varsayımı ve Varyans Homojenliğidir.

In [23]:
# Normallik Varsayımının hipotezini kuralım;

# H0: Normal dağılım varsayımı sağlanmaktadır.
# H1: Normal dağılım varsayımı sağlanmamaktadır.

# p < 0.05 H0 RED
# p > 0.05 H0 REDDEDİLEMEZ

test_stat, pvalue = shapiro(df.loc[df["group"] == "control", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9773, p-value = 0.5891


    HO reddedilemez. Control grubunun değerleri normal dağılım varsayımını sağlamaktadır.

In [25]:
test_stat, pvalue = shapiro(df.loc[df["group"] == "test", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9589, p-value = 0.1541


    HO reddedilemez. Test grubunun da değerleri normal dağılım varsayımını sağlamaktadır.

In [24]:
# Varyans Homojenliğinin hipotez testlerini kuralım;

# H0: Varyanslarhomojendir.
# H1: Varyanslarhomojen Değildir.

# p < 0.05 H0 RED
# p > 0.05 H0 REDDEDİLEMEZ

test_stat, pvalue = levene(df.loc[df["group"] == "control", "Purchase"],
                           df.loc[df["group"] == "test", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 2.6393, p-value = 0.1083


    HO reddedilemez. Control ve Test grubunun değerleri varyans homejenliği varsayımını sağlamaktadır.
    Varyanslar Homojendir.

In [26]:
# Varsayımlar sağlandığı için bağımsız iki örneklem t testi (parametrik test) yapacağız.

# T- testinin hipatezlerini yazalım;

# H0: M1 = M2 (Kontrol grubu ve test grubu satın alma ortalamaları arasında ist. ol.anl.fark yoktur.)
# H1: M1 != M2 (Kontrol grubu ve test grubu satın alma ortalamaları arasında ist. ol.anl.fark vardır)

# p<0.05 HO RED 
# p>0.05 HO REDDEDİLEMEZ

test_stat, pvalue = ttest_ind(df.loc[df["group"] == "control", "Purchase"],
                              df.loc[df["group"] == "test", "Purchase"],
                              equal_var=True)

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = -0.9416, p-value = 0.3493


    HO reddedilemez. 
    Kontrol ve test grubu satın alma ortalamaları arasında istatistiksel olarak anlamlı farklılık yoktur.